In [1]:
from sagemaker.huggingface import HuggingFace
import sagemaker
import os

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/abhijitdeshpande/Library/Application Support/sagemaker/config.yaml


In [2]:
ROLE='arn:aws:iam::720332985926:role/service-role/AmazonSageMaker-ExecutionRole-20240705T185967'
SOURCE_DIR='/Volumes/LaCie/Projects_portfolio/NLP/SupportIQ/src'

In [3]:
os.environ['AWS_PROFILE'] = 'dev-user'

In [5]:
sess = sagemaker.Session()

In [7]:
hyper_parameters = {
    'model_id': 'google/flan-t5-base',
    'rank': 8,
    'alpha': 16,
    'dropout': 0.05,
    'bias': 'none',
    'lr': 1e-5,
    'epochs': 1,
    'wd': 0.01,
    'logging_steps': 1,
    'batch_size': 32
}

In [8]:
huggingface_estimator = HuggingFace(
            entry_point='train.py',
            source_dir=SOURCE_DIR,
            role=ROLE,
            instance_type='ml.g4dn.xlarge',
            instance_count=1,
            transformers_version='4.49.0',
            py_version='py311',
            pytorch_version='2.5.1',
            hyperparameters = hyper_parameters
)

In [7]:
# upload data to S3
input_data = sess.upload_data(path='data/data_full.json',bucket='gen-ai-repository', key_prefix='finetuning/flan-t5/data')

In [9]:
huggingface_estimator.fit({'training': 's3://gen-ai-repository/finetuning/flan-t5/data/'})

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: huggingface-pytorch-training-2025-06-19-23-24-05-737


2025-06-19 23:24:09 Starting - Starting the training job...
2025-06-19 23:24:24 Starting - Preparing the instances for training...
2025-06-19 23:24:55 Downloading - Downloading input data...
2025-06-19 23:25:20 Downloading - Downloading the training image...........................
2025-06-19 23:30:13 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
CUDA compat package should be installed for NVIDIA driver smaller than 550.144.03
Current installed NVIDIA driver version is 550.163.01
Skipping CUDA compat setup as newer NVIDIA driver is installed
2025-06-19 23:30:25,565 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2025-06-19 23:30:25,586 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2025-06-19 23:30:25,596 sagemaker_pytorch_container.training INFO     Block until all hos

In [10]:
model_artifact_s3_uri = huggingface_estimator.model_data
print("Model Artifact S3 URI:", model_artifact_s3_uri)

Model Artifact S3 URI: s3://sagemaker-us-east-1-720332985926/huggingface-pytorch-training-2025-06-19-23-24-05-737/output/model.tar.gz
